## Connect to database file

In [74]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [75]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Tables and views on database

In [76]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Here is also the database schema 

![Here is also the database schema](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

## Genres that sell most tracks in the US

In [77]:
%%sql

WITH tracks_sold_in_USA AS
     (
      SELECT il.* 
      FROM invoice_line il
      INNER JOIN invoice i ON i.invoice_id = il.invoice_id
      INNER JOIN customer c ON c.customer_id = i.customer_id
      WHERE c.country = "USA"
     )


SELECT
    g.name genre,
    count(tusa.invoice_line_id) tracks_sold,
    round(cast(count(tusa.invoice_line_id) AS FLOAT) / (
     SELECT COUNT(*) 
            FROM tracks_sold_in_USA)
          ,4)
     percentage_sold
FROM tracks_sold_in_usa tusa
INNER JOIN track t on t.track_id = tusa.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC;


Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5338
Alternative & Punk,130,0.1237
Metal,124,0.118
R&B/Soul,53,0.0504
Blues,36,0.0343
Alternative,35,0.0333
Latin,22,0.0209
Pop,22,0.0209
Hip Hop/Rap,20,0.019
Jazz,14,0.0133


The three arists are chosen based on the most sold genres in the US. 
 1st - Red Tone - Punk - Percentage of genre sold in US: 53.38%
 2nd - Slim Jim Bites - Blues - Percentage of genre sold in US: 3.43%
 3rd - Meteor and the Girls - Pop - Percentage of genre sold in US: 2.09%

### Total amount of sales by sales support agent

In [78]:
%%sql
SELECT 
e.employee_id ID,
e.first_name || " " || e.last_name as Name,
e.country as Country,
e.hire_date,
ROUND(sum(i.total),2) total_sales
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
group by 2;

Done.


ID,Name,Country,hire_date,total_sales
3,Jane Peacock,Canada,2017-04-01 00:00:00,1731.51
4,Margaret Park,Canada,2017-05-03 00:00:00,1584.0
5,Steve Johnson,Canada,2017-10-17 00:00:00,1393.92


Looking only and the total_sales column, we can see that Jane is the agent with the biggest amount of sales. But the hiring date allows us to correctly interpret these results. 

While her and Margaret may have the same performance (Jane's sales 9.3% bigger than Margaret's with the latter entering 1 month later the company) Steve has proven to be the best performer. That is because even though he has 24% less of sales than Jane, he entered the company 6 months later! 

### Analyze Sales by Country

In [79]:
%%sql

WITH country_or_other AS
    (
     SELECT CASE 
           WHEN (SELECT COUNT(*) 
                        FROM customer
                        WHERE country = c.country) = 1 THEN "Other"
           ELSE c.country
        END AS country,
        c.customer_id,
        il.*
        FROM customer c 
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
     )

SELECT country,
       customers,
       total_sales,
       avg_sales_customer,
       avg_order_value
    FROM (SELECT country,
                 count(distinct customer_id) customers,
                 sum(unit_price) total_sales,
                 sum(unit_price)/count(customer_id) avg_sales_customer,
                 sum(unit_price)/count(invoice_id) avg_order_value,
                 CASE 
                     WHEN country = "Other" THEN 1
                     ELSE 0
                     END AS sort
          FROM country_or_other
    GROUP BY 1
    ORDER BY sort ASC, total_sales DESC
    );
    
                



                

Done.


country,customers,total_sales,avg_sales_customer,avg_order_value
USA,13,1040.490000000008,0.9900000000000075,0.9900000000000075
Canada,8,535.5900000000034,0.9900000000000063,0.9900000000000063
Brazil,5,427.68000000000245,0.9900000000000057,0.9900000000000057
France,5,389.0700000000021,0.9900000000000053,0.9900000000000053
Germany,4,334.6200000000016,0.9900000000000048,0.9900000000000048
Czech Republic,2,273.24000000000103,0.9900000000000038,0.9900000000000038
United Kingdom,3,245.52000000000078,0.9900000000000031,0.9900000000000031
Portugal,2,185.13000000000022,0.9900000000000012,0.9900000000000012
India,2,183.1500000000002,0.9900000000000011,0.9900000000000011
Other,15,1094.9400000000085,0.9900000000000077,0.9900000000000077


### Number and percentage of album purchases

In [80]:
%%sql 
WITH invoice_first_track AS
     (
      SELECT invoice_id,
             MIN(track_id) first_track_id
             FROM invoice_line
         GROUP BY 1
     )
    

SELECT album_purchase,
       COUNT(invoice_id) total_invoices,
       CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*)
                            FROM invoice) percent
        FROM
        (SELECT ift.*,
                CASE
                    WHEN (   SELECT t.track_id FROM track t
                             WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                             WHERE t2.track_id = ift.first_track_id)
                         
          
                       EXCEPT 
                              SELECT il.track_id FROM invoice_line il
                              WHERE il.invoice_id = ift.invoice_id
                          ) IS NULL
                    AND
                        (      SELECT il.track_id FROM invoice_line il
                               WHERE il.invoice_id = ift.invoice_id 
                    
                         EXCEPT
                         
                            SELECT t.track_id FROM track t
                            WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                             WHERE t2.track_id = ift.first_track_id)
                         ) IS NULL
                    THEN "Yes"
                    ELSE "No"
                END AS album_purchase
           FROM invoice_first_track ift)
         GROUP BY album_purchase

Done.


album_purchase,total_invoices,percent
No,500,0.8143322475570033
Yes,114,0.18566775244299674


I wouldn't advice to remove full album purchases since full-album purchases represent almost 20% of invoice.

### Most used artist in playlists

In [81]:
%%sql

SELECT ar.name,
       COUNT(distinct pt.playlist_id) number_of_playlists,
       COUNT(distinct pt.track_id) number_of_tracks
FROM playlist_track pt
INNER JOIN track t ON t.track_id = pt.track_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
GROUP BY 1
ORDER BY 2 DESC, 3 DESC


Done.


name,number_of_playlists,number_of_tracks
Eugene Ormandy,7,3
Berliner Philharmoniker & Herbert Von Karajan,6,3
Academy of St. Martin in the Fields & Sir Neville Marriner,6,2
English Concert & Trevor Pinnock,6,2
The King's Singers,6,2
Michael Tilson Thomas & San Francisco Symphony,5,2
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",5,1
Adrian Leaper & Doreen de Feis,5,1
Alberto Turco & Nova Schola Gregoriana,5,1
Antal Doráti & London Symphony Orchestra,5,1


### Tracks purchased vs non purchased

In order to do this we need to now: 
* Number of total tracks in the app --> total_tracks
* Number of distinct tracks in invoice_line --> tracks_purchased

In [82]:
%%sql

SELECT (SELECT count(distinct track_id) FROM invoice_line) tracks_purchased,
       count(distinct track_id) - (SELECT count(distinct track_id) FROM invoice_line) tracks_non_purchaded,
       ROUND((SELECT count(distinct track_id) FROM invoice_line)/CAST(count(distinct track_id) AS FLOAT), 3) *100 percentage,
       count(distinct track_id) total_tracks
FROM track


Done.


tracks_purchased,tracks_non_purchaded,percentage,total_tracks
1806,1697,51.6,3503


51.6% of the tracks available have been sold.

### Protected vs non-protected media types. Effect on sales. 

* Tenemos que crearnos una tabla donde estén todas las tracks compradas diciendo si son protected o non-protected. Para ello clasificamos los 5 tipos de media_type_id
 1,4,5 - NO PROTECTED
 2,3 - PROTECTED 
 
* A partir de ahí solo tenemos que sumar las ventas cada parte aplicando un WHERE para distinguir las dos posibilidades y sacar los porcentajes

In [83]:
%%sql 

WITH protected_track_sales AS
     (
         SELECT COUNT(il.track_id) protected_tracks_bought,
                SUM(il.unit_price) protected_sales
        FROM invoice_line il 
        INNER JOIN track t on t.track_id = il.track_id
        INNER JOIN media_type mt on mt.media_type_id = t.media_type_id
        WHERE mt.media_type_id = 2 
              OR mt.media_type_id = 3 

        ORDER BY 1 ASC
         
     )
    
SELECT (SELECT protected_tracks_bought FROM protected_track_sales) protected_tracks_sold,
       (SELECT protected_sales FROM protected_track_sales) protected_sales,
        SUM(unit_price) total_sales,
       (SELECT protected_sales FROM protected_track_sales)/SUM(unit_price) * 100 percentage
    
 FROM invoice_line    
 
       

Done.


protected_tracks_sold,protected_sales,total_sales,percentage
442,437.58000000000254,4709.429999999431,9.291570317428127


Protected sales are not really popular, representing roughly 9.30% of total sales. 